In [ ]:
from google.colab import drive
drive.mount('/content/drive')
import pandas as pd
import numpy as np
import openpyxl
from geopy.distance import geodesic
from math import sin, cos, radians
import seaborn as sns
from sklearn.cluster import DBSCAN
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import silhouette_score, calinski_harabasz_score
from sklearn.cluster import KMeans
from scipy.spatial.distance import cdist
import matplotlib.pyplot as plt
from sklearn.metrics import silhouette_score
from shapely.geometry import Polygon
from sklearn.metrics import pairwise_distances
from sklearn.metrics.pairwise import pairwise_distances_argmin_min
from geopy.distance import great_circle
from tqdm import tqdm
tqdm.pandas()

In [ ]:
# 데이터프레임의 열 이름 변경
fire = fire.rename(columns={'지역':'시군구'})
road = road.rename(columns={'lat':'위도','long':'경도'})
wind = wind.rename(columns={'일시':'DATE'})
kor = kor.rename(columns={'latitude':'위도','longitude':'경도'})
kor1 = kor1.rename(columns={'latitude':'위도','longitude':'경도'})
kor1['시군구'] = kor1['do'] + ' ' + kor1['city']
wind.drop(['Unnamed: 0'],axis=1,inplace=True)

In [ ]:
# 각 데이터셋에서 고유한 날짜를 가져옴
unique_dates1 = set(fire['DATE'])
unique_dates2 = set(road['DATE'])
unique_dates3 = set(wind['DATE'])

all_unique_dates = sorted(unique_dates1.union(unique_dates2).union(unique_dates3))

expanded_rows = []

# 'kor' 데이터프레임의 각 행을 반복하면서 각 고유 날짜에 대해 새로운 행을 생성
for _, row in kor.iterrows():
    for date in all_unique_dates:
        new_row = row.copy()
        new_row['DATE'] = date
        expanded_rows.append(new_row)

kor = pd.DataFrame(expanded_rows)

In [ ]:
# 'kor'을 기준으로 각 데이터셋을 'DATE'와 '시군구' 열을 사용해 병합
fire_ko = pd.merge(kor, fire, how='outer',on=['DATE','시군구'])
road_ko = pd.merge(kor, road, how='outer',on=['DATE','시군구'])
wind_ko = pd.merge(kor, wind, how='outer',on=['DATE','시군구'])

fire_ko = fire_ko.rename(columns={'위도_y':'위도_화력','경도_y':'경도_화력',
                                  '위도_x':'위도_행정','경도_x':'경도_행정'})
road_ko = road_ko.rename(columns={'위도_y':'위도_도로','경도_y':'경도_도로',
                                  '위도_x':'위도_행정','경도_x':'경도_행정'})
wind_ko = wind_ko.rename(columns={'위도_y':'위도_풍향','경도_y':'경도_풍향',
                                  '위도_x':'위도_행정','경도_x':'경도_행정'})
fire_ko.drop(['발전소 이름'],axis=1,inplace=True)
road_ko.drop(['도로명','시작점','종점','측정거리(km)','상태'],axis=1,inplace=True)
wind_ko.drop(['관리관서','지점번호','최대풍속(m/s)','최대순간풍속풍향(deg)','최대순간풍속시각',
              '최대순간풍속풍향(deg)','최대순간풍속(m/s)'],axis=1,inplace=True)

In [ ]:
merged = pd.merge(wind_ko,road_ko,how='outer',on=['DATE','시군구'])
merged.drop(['경도_행정_y','위도_행정_y','지점주소','지점명_x'],axis=1,inplace=True)
merged = merged.rename(columns={'위도_행정_x':'위도_행정','경도_행정_x':'경도_행정'})
merged = pd.merge(merged,fire_ko,how='outer',on=['DATE','시군구'])
merged = merged.rename(columns={'위도_행정_x':'위도_행정','경도_행정_x':'경도_행정'})
merged.drop(['경도_행정_y','위도_행정_y','최대풍속시각'],axis=1,inplace=True)

In [ ]:
# 각 데이터셋에서 관련된 모든 좌표를 결합하여 클러스터링 정보 생성
cluster_info = pd.concat([kor[['위도', '경도']],
                         wind[['위도', '경도']],
                         road[['위도', '경도']],
                         fire[['위도', '경도']]])
cluster_info = cluster_info.drop_duplicates().reset_index(drop=True)

kmeans = KMeans(n_clusters=5, init='k-means++', random_state=42)
cluster_info['cluster'] = kmeans.fit_predict(cluster_info)

lat_lon_columns = [
    ('위도_행정', '경도_행정'),
    ('위도_풍향', '경도_풍향'),
    ('위도_도로', '경도_도로'),
    ('위도_화력', '경도_화력')
]

new_data = pd.DataFrame()

for lat_col, lon_col in lat_lon_columns:
    temp_df = merged[[lat_col, lon_col]].dropna()
    temp_df.columns = ['위도', '경도']
    new_data = pd.concat([new_data, temp_df])

new_data = new_data.drop_duplicates().reset_index(drop=True)

kmeans = KMeans(n_clusters=5, init='k-means++', random_state=42)
new_data['cluster'] = kmeans.fit_predict(new_data)

In [ ]:
df = df_17

# 필요한 컬럼 이름 리스트
columns_common = ['시군구', 'DATE', '평균풍속(m/s)', '최대풍속풍향(deg)', '기온(℃)', '습도(%)', '재비산먼지 평균농도(㎍/㎥)', '연료원', '발전량(MWh)', '용량(MW)', 'cluster']

# 새로운 데이터프레임을 저장할 리스트
new_rows = []

# 각 위치 유형(행정, 풍향, 도로, 화력)에 대해 위도/경도가 비어 있지 않으면 새로운 행 생성
for idx, row in tqdm(df.iterrows(), total=df.shape[0], desc="Processing rows"):
    # 행정 위치에 대한 행
    if not pd.isnull(row['경도_행정']) and not pd.isnull(row['위도_행정']):
        new_row = [row['경도_행정'], row['위도_행정']] + [row[col] for col in columns_common]
        new_rows.append(new_row)

    # 풍향 위치에 대한 행
    if not pd.isnull(row['경도_풍향']) and not pd.isnull(row['위도_풍향']):
        new_row = [row['경도_풍향'], row['위도_풍향']] + [row[col] for col in columns_common]
        new_rows.append(new_row)

    # 도로 위치에 대한 행
    if not pd.isnull(row['경도_도로']) and not pd.isnull(row['위도_도로']):
        new_row = [row['경도_도로'], row['위도_도로']] + [row[col] for col in columns_common]
        new_rows.append(new_row)

    # 화력 위치에 대한 행
    if not pd.isnull(row['경도_화력']) and not pd.isnull(row['위도_화력']):
        new_row = [row['경도_화력'], row['위도_화력']] + [row[col] for col in columns_common]
        new_rows.append(new_row)

new_columns = ['경도', '위도'] + columns_common
new_df = pd.DataFrame(new_rows, columns=new_columns)


columns_to_fill = ['발전량(MWh)', '용량(MW)', '재비산먼지 평균농도(㎍/㎥)', '평균풍속(m/s)','최대풍속풍향(deg)','연료원']


# 역거리 가중치를 사용하여 클러스터 내 결측값을 채우는 함수.
def inverse_distance_weighting(df, target_col, cluster_col='cluster', lat_col='위도_행정', lon_col='경도_행정'):
    clusters = df[cluster_col].unique()
    filled_values = []

    for cluster in tqdm(clusters, desc=f'Processing {target_col}', leave=False):
        cluster_data = df[df[cluster_col] == cluster]
        known_values = cluster_data.dropna(subset=[target_col])
        missing_values = cluster_data[cluster_data[target_col].isnull()]

        if not known_values.empty and not missing_values.empty:
            known_coords = known_values[[lat_col, lon_col]].values
            missing_coords = missing_values[[lat_col, lon_col]].values

            distances = cdist(missing_coords, known_coords, 'euclidean')
            weights = 1 / (distances + 1e-10)  # 작은 값을 더해 안정화

            # missing_values의 인덱스를 재설정하고, 기존 인덱스를 저장합니다.
            missing_values = missing_values.reset_index()
            missing_values.rename(columns={'index': 'original_index'}, inplace=True)

            for idx, row in missing_values.iterrows():
                weights_sum = np.sum(weights[idx])
                if weights_sum == 0:
                    continue
                if np.issubdtype(known_values[target_col].dtype, np.number):
                    weighted_avg = np.sum(weights[idx] * known_values[target_col].values) / weights_sum
                    original_index = row['original_index']
                    df.at[original_index, target_col] = weighted_avg
                    filled_values.append((original_index, weighted_avg))
                else:
                    # 비숫자 열 처리 (예: 가장 빈번한 값 찾기)
                    most_frequent_value = known_values[target_col].mode()[0]
                    original_index = row['original_index']
                    df.at[original_index, target_col] = most_frequent_value
                    filled_values.append((original_index, most_frequent_value))

        elif not missing_values.empty:
            if np.issubdtype(df[target_col].dtype, np.number):
                global_mean = df[target_col].mean()
            else:
                # 숫자가 아닌 열의 경우 가장 빈번한 값 가져오기
                global_mean = df[target_col].mode()[0]

            if pd.isnull(global_mean):
                continue
            missing_values = missing_values.reset_index()
            missing_values.rename(columns={'index': 'original_index'}, inplace=True)
            for idx, row in missing_values.iterrows():
                original_index = row['original_index']
                df.at[original_index, target_col] = global_mean
                filled_values.append((original_index, global_mean))

    return df, filled_values

for col in columns_to_fill:
    df_17, filled_values = inverse_distance_weighting(df_17, col)